# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [1]:
import torch
from torch import nn
from torchvision import models
from collections import OrderedDict

# Load your model to this variable
# TODO: Write a function that loads a checkpoint and rebuilds the model
flatten_layers = {"vgg16":25088,
                  "densenet121":1024}

def get_model(name="vgg16", classifier=None, drop_prob=0.5):
    """Get pre trained model and stack fully connected layers
    
    Arguments
    ---------
    name -> pre trained model name (vgg16 or densenet121)
    classifier -> nn.Sequencial classifier. If None, a default classifier will be set in place.
    drop_prob -> dropout value
    """
    
    # load pre trained model
    if name == "vgg16":
        model = models.vgg16(pretrained=True)
    elif name == "densenet121":
        model = models.densenet121(pretrained=True)
    else:
        print("Model not available at the moment.")
        
    # freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
        
    # build classifier layers
    if classifier is None:
        classifier = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(flatten_layers[name], 1024)),
                                ('relu1', nn.ReLU()),
                                ('drop1', nn.Dropout(drop_prob)),
                                ('fc2', nn.Linear(1024, 512)),
                                ('relu2', nn.ReLU()),
                                ('drop2', nn.Dropout(drop_prob)),
                                ('fc3', nn.Linear(512, 102)),
                                ('output', nn.LogSoftmax(dim=1))
                                ]))
    
    # add classifier to model
    model.classifier = classifier
    # model to gpu
    #model.cuda()
    
    return model 

def load_model(path):
    """Load checkpoints model from path"""
  
    checkpoint = torch.load(path, map_location=lambda storage, loc: storage)
  
    name = checkpoint["name"]
    classifier = checkpoint["classifier"]
    drop_prob = checkpoint["drop_prob"]
  
    model = get_model(name, classifier, drop_prob)
    model.class_to_idx = checkpoint['class_to_idx']
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    model.eval()
  
    return model
  
model = load_model("final_project_densenet121.pth")
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
